In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200828


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [5]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_オンライン入会分析")

In [6]:
#input
input_new = "NEW_CUST_List"
input_uri = "NEW_CUST_SUM_SHOPPING_LIST"

code_list = "code-todoufuken"
code_list = pd.read_excel(data_dir /f"{code_list}.xlsx",encoding='shift-jis')

kameiten_list = pd.read_excel(data_dir /f"代表加盟店名称201912.xls",encoding='shift-jis')
kameiten_list = kameiten_list.rename(columns={'代表加盟店番号':'daihyokameiten_no',
                                             '代表加盟店名':'daihyokameiten_name'})


In [7]:
kameiten_list.head()

,daihyokameiten_no,daihyokameiten_name,代表業種コード,代表業種名
0,100051669,シューズスクエア ススキノ店,301.0,靴
1,100051723,Ｗ・ＩブティックＥＸ,302.0,鞄
2,100077862,ホームセンターハッピー１,299.0,その他用品
3,100078087,ブティック なみ木,199.0,その他百貨店
4,100078109,おしゃれ工房 彩,202.0,婦人服


In [11]:
df_new = pd.read_csv(data_dir /f"{input_new}.csv",encoding='shift-jis')
df_new.head(2)

,cp_date,no,kanyu_date,card,channel,kzk_f,age,gender,income,prefecture,job,house,email_f,web_f
0,20190331,4980006396956008,20190301,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0
1,20190331,4980006396965009,20190301,ゴールド,店頭タブレット,1,55〜59歳,男性,500万円以上〜1，000万円未満,兵庫県,0,0,0,0


In [12]:
#日付データの変換
df_new['cp_date']= pd.to_datetime(df_new['cp_date'], format='%Y%m%d')
df_new['kanyu_date'] = pd.to_datetime(df_new['kanyu_date'], format='%Y%m%d')

df_new['kanyu_year'] = df_new['kanyu_date'].dt.year
df_new['kanyu_month'] = df_new['kanyu_date'].dt.month

df_new['kanyu_yymm'] = df_new[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_new.head(2)

,cp_date,no,kanyu_date,card,channel,kzk_f,age,gender,income,prefecture,job,house,email_f,web_f,kanyu_year,kanyu_month,kanyu_yymm
0,2019-03-31,4980006396956008,2019-03-01,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0,2019,3,2019年3月
1,2019-03-31,4980006396965009,2019-03-01,ゴールド,店頭タブレット,1,55〜59歳,男性,500万円以上〜1，000万円未満,兵庫県,0,0,0,0,2019,3,2019年3月


In [14]:
#df_new['cp_date'] = df_new['cp_date'].dt.strftime('%Y/%m/%d')
#df_new['kanyu_date'] = df_new['kanyu_date'].dt.strftime('%Y/%m/%d')

In [15]:
#性・年齢別
q = """
        SELECT
        kanyu_yymm
        ,card
        ,channel
        ,age
        ,gender
        ,count(no) as no_of_new_account
        from 
            df_new
        group by
            1,2,3,4,5
        """

df_age = sqldf(q, locals())
df_age.head()

,kanyu_yymm,card,channel,age,gender,no_of_new_account
0,2019年10月,ゴールド,オンライン,25〜29歳,女性,1
1,2019年10月,ゴールド,オンライン,25〜29歳,男性,1
2,2019年10月,ゴールド,オンライン,30〜34歳,女性,3
3,2019年10月,ゴールド,オンライン,30〜34歳,男性,3
4,2019年10月,ゴールド,オンライン,35〜39歳,男性,5


In [16]:
print(df_age['kanyu_yymm'].unique())

['2019年10月' '2019年11月' '2019年12月' '2019年3月' '2019年4月' '2019年5月' '2019年6月'
 '2019年7月' '2019年8月' '2019年9月' '2020年1月' '2020年2月' '2020年3月' '2020年4月'
 '2020年5月' '2020年6月']


In [18]:
#CSV出力
df_age.to_csv(data_dir /r"output"/'new_cust_by_age.csv', header=True ,index=None, encoding='shift=jis')

In [19]:
#年収別
q = """
        SELECT
        kanyu_yymm
        ,card
        ,channel
        ,income
        ,count(no) as no_of_new_account
        from 
            df_new
        group by
            1,2,3,4
        """

df_income = sqldf(q, locals())
df_income.head()

,kanyu_yymm,card,channel,income,no_of_new_account
0,2019年10月,ゴールド,オンライン,"1,000万円以上",15
1,2019年10月,ゴールド,オンライン,100万円以上〜200万円未満,1
2,2019年10月,ゴールド,オンライン,300万円以上〜400万円未満,2
3,2019年10月,ゴールド,オンライン,400万円以上〜500万円未満,2
4,2019年10月,ゴールド,オンライン,500万円以上〜1，000万円未満,14


In [20]:
#CSV出力
df_income.to_csv(data_dir /r"output"/'new_cust_by_income.csv', header=True ,index=None, encoding='shift=jis')

In [25]:
df_new = pd.merge(df_new,code_list, on =['prefecture'], how = 'left')

In [26]:
#都道府県別
q = """
        SELECT
        kanyu_yymm
        ,card
        ,channel
        ,prefecture2
        ,count(no) as no_of_new_account
        from 
            df_new
        group by
            1,2,3,4
        """

df_pref = sqldf(q, locals())
df_pref.head()

,kanyu_yymm,card,channel,prefecture2,no_of_new_account
0,2019年10月,ゴールド,オンライン,01_北海道,4
1,2019年10月,ゴールド,オンライン,12_千葉県,1
2,2019年10月,ゴールド,オンライン,13_東京都,5
3,2019年10月,ゴールド,オンライン,18_福井県,1
4,2019年10月,ゴールド,オンライン,21_岐阜県,1


In [27]:
#CSV出力
df_pref.to_csv(data_dir /r"output"/'new_cust_by_pref.csv', header=True ,index=None, encoding='shift=jis')

In [28]:
#職業別
q = """
        SELECT
        kanyu_yymm
        ,card
        ,channel
        ,job
        ,count(no) as no_of_new_account
        from 
            df_new
        group by
            1,2,3,4
        """

df_job = sqldf(q, locals())
df_job.head()

,kanyu_yymm,card,channel,job,no_of_new_account
0,2019年10月,ゴールド,オンライン,1,7
1,2019年10月,ゴールド,オンライン,3,25
2,2019年10月,ゴールド,オンライン,5,1
3,2019年10月,ゴールド,オンライン,9,1
4,2019年10月,ゴールド,店頭タブレット,0,163


In [29]:
#CSV出力
df_job.to_csv(data_dir /r"output"/'new_cust_by_job.csv', header=True ,index=None, encoding='shift=jis')

In [30]:
#住居別
q = """
        SELECT
        kanyu_yymm
        ,card
        ,channel
        ,house
        ,count(no) as no_of_new_account
        from 
            df_new
        group by
            1,2,3,4
        """

df_house = sqldf(q, locals())
df_house.head()

,kanyu_yymm,card,channel,house,no_of_new_account
0,2019年10月,ゴールド,オンライン,0,8
1,2019年10月,ゴールド,オンライン,1,1
2,2019年10月,ゴールド,オンライン,6,8
3,2019年10月,ゴールド,オンライン,9,1
4,2019年10月,ゴールド,オンライン,A,7


In [31]:
#CSV出力
df_house.to_csv(data_dir /r"output"/'new_cust_by_house.csv', header=True ,index=None, encoding='shift=jis')

売上データ 入会3カ月以内の利用加盟店Top20

In [13]:
df_uri = pd.read_csv(data_dir /f"{input_uri}.csv",encoding='shift-jis')
df_uri.head(2)

,cp_date,no,credit_date,kameiten_no,daihyokameiten_no,gyoshu_code,gyoshumei,chokuei_code,meti_code,shoping_amt
0,20190331,4980006396956008,21619,0,0,3701,OCAP 他ブランド利用,*,0316_OCAP他ブランド,4655
1,20190331,4980006396956008,21621,0,0,3701,OCAP 他ブランド利用,*,0316_OCAP他ブランド,4284


In [14]:
df_uri.shape[0]

1275060

In [15]:
df_new2 = df_new.drop(columns={'cp_date'})
df_uri = pd.merge(df_uri,df_new2, on =['no'], how = 'left')
df_uri.head(2)

,cp_date,no,credit_date,kameiten_no,daihyokameiten_no,gyoshu_code,gyoshumei,chokuei_code,meti_code,shoping_amt,kanyu_date,card,channel,kzk_f,age,gender,income,prefecture,job,house,email_f,web_f,kanyu_year,kanyu_month,kanyu_yymm
0,20190331,4980006396956008,21619,0,0,3701,OCAP 他ブランド利用,*,0316_OCAP他ブランド,4655,2019-03-01,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0,2019,3,2019年3月
1,20190331,4980006396956008,21621,0,0,3701,OCAP 他ブランド利用,*,0316_OCAP他ブランド,4284,2019-03-01,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0,2019,3,2019年3月


In [16]:
df_uri.shape[0]

1275060

In [17]:
df_uri = pd.merge(df_uri,kameiten_list, on =['daihyokameiten_no'], how = 'left')
df_uri.head()

,cp_date,no,credit_date,kameiten_no,daihyokameiten_no,gyoshu_code,gyoshumei,chokuei_code,meti_code,shoping_amt,kanyu_date,card,channel,kzk_f,age,gender,income,prefecture,job,house,email_f,web_f,kanyu_year,kanyu_month,kanyu_yymm,daihyokameiten_name,代表業種コード,代表業種名
0,20190331,4980006396956008,21619,0,0,3701,OCAP 他ブランド利用,*,0316_OCAP他ブランド,4655,2019-03-01,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0,2019,3,2019年3月,NaN,NaN,NaN
1,20190331,4980006396956008,21621,0,0,3701,OCAP 他ブランド利用,*,0316_OCAP他ブランド,4284,2019-03-01,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0,2019,3,2019年3月,NaN,NaN,NaN
2,20190331,4980006396956008,21624,0,0,3701,OCAP 他ブランド利用,*,0316_OCAP他ブランド,3489,2019-03-01,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0,2019,3,2019年3月,NaN,NaN,NaN
3,20190331,4980006396956008,21626,0,0,3701,OCAP 他ブランド利用,*,0316_OCAP他ブランド,2207,2019-03-01,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0,2019,3,2019年3月,NaN,NaN,NaN
4,20190331,4980006396956008,21635,199728038,199728038,101,百貨店,直営店,0311_百貨店,1166,2019-03-01,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0,2019,3,2019年3月,大丸京都店,101.0,百貨店


In [18]:
df_uri['cp_date']= pd.to_datetime(df_uri['cp_date'], format='%Y%m%d')

In [19]:
#経過月数
df_uri['mob']=(df_uri['cp_date'].dt.year - df_uri['kanyu_date'].dt.year) * 12 + df_uri['cp_date'].dt.month - df_uri['kanyu_date'].dt.month
df_uri.head(2)

,cp_date,no,credit_date,kameiten_no,daihyokameiten_no,gyoshu_code,gyoshumei,chokuei_code,meti_code,shoping_amt,kanyu_date,card,channel,kzk_f,age,gender,income,prefecture,job,house,email_f,web_f,kanyu_year,kanyu_month,kanyu_yymm,daihyokameiten_name,代表業種コード,代表業種名,mob
0,2019-03-31,4980006396956008,21619,0,0,3701,OCAP 他ブランド利用,*,0316_OCAP他ブランド,4655,2019-03-01,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0,2019,3,2019年3月,NaN,NaN,NaN,0
1,2019-03-31,4980006396956008,21621,0,0,3701,OCAP 他ブランド利用,*,0316_OCAP他ブランド,4284,2019-03-01,ゴールド,店頭タブレット,0,70〜74歳,女性,400万円以上〜500万円未満,京都府,9,A,0,0,2019,3,2019年3月,NaN,NaN,NaN,0


In [20]:
print(df_uri['mob'].unique())

[ 0  1  2  3 -1  4  5  6  7  8  9 10 11 12 13 14 15]


In [21]:
#2020年1月以降入会客、百貨店と外部加盟店の内訳
q = """
        SELECT
        channel
        ,chokuei_code
        ,count(no) as no_of_trn
        ,sum(shoping_amt) as sum_shopping_amt
        from 
            df_uri
        where 
            kanyu_year = 2020
            and kanyu_month >= 1
        group by
        1,2
        """

df_choku = sqldf(q, locals())
df_choku

,channel,chokuei_code,no_of_trn,sum_shopping_amt
0,オンライン,*,7364,57055260
1,オンライン,直営店,1614,42506406
2,オンライン,関係百貨店,27,526585
3,店頭タブレット,*,35679,321378465
4,店頭タブレット,直営店,12911,230241168
5,店頭タブレット,関係百貨店,90,845565
6,郵送,*,66336,775210980
7,郵送,直営店,22157,813234527
8,郵送,関係百貨店,773,15001349


In [22]:
#CSV出力
df_choku.to_csv(data_dir /r"output"/'new_cust_choku_gaibu.csv', header=True ,index=None, encoding='shift=jis')

In [69]:
#2020年1月以降入会客、外部加盟店での利用状況
q = """
        SELECT
        daihyokameiten_name
        ,meti_code
        ,gyoshumei
        ,channel
        ,count(no) as no_of_trn
        ,sum(shoping_amt) as sum_shopping_amt
        from 
            df_uri
        where 
            kanyu_year = 2020
            and kanyu_month >= 1
        and chokuei_code in ('*')
        group by
        1,2,3,4
        """

df_shop = sqldf(q, locals())
df_shop

,daihyokameiten_name,meti_code,gyoshumei,channel,no_of_trn,sum_shopping_amt
0,None,0311_百貨店,ショッピングセンター・コンビニほか,オンライン,38,141274
1,None,0311_百貨店,ショッピングセンター・コンビニほか,店頭タブレット,209,1318814
2,None,0311_百貨店,ショッピングセンター・コンビニほか,郵送,369,2524709
3,None,0311_百貨店,百貨店,オンライン,2,4070
4,None,0311_百貨店,百貨店,店頭タブレット,14,63089
...,...,...,...,...,...,...
3929,ｕｍｉｅ ＮＯＲＴＨＭＡＬＬ,0311_百貨店,ショッピングセンター・コンビニほか,店頭タブレット,4,12705
3930,ｕｍｉｅ ＮＯＲＴＨＭＡＬＬ,0311_百貨店,ショッピングセンター・コンビニほか,郵送,3,26954
3931,ｕｎｉｃｏ 福岡,0312_スーパー・コンビニ・小売り,家具・建具・什器小売,店頭タブレット,1,76450
3932,ﾌﾟﾘﾍﾟｲﾄﾞ購入・ﾁｬｰｼﾞ（SMCC）,0316_OCAP他ブランド,電子商取引（SSL）,オンライン,2,1000


In [71]:
q = """
        SELECT
        daihyokameiten_name
        ,meti_code
        ,gyoshumei
        ,channel
        ,no_of_trn
        ,sum_shopping_amt
        ,row_number() over(partition by channel order by no_of_trn desc) as rank
        from 
            df_shop
        order by
        channel,rank
        """

df_shop2 = sqldf(q, locals())
df_shop2

,daihyokameiten_name,meti_code,gyoshumei,channel,no_of_trn,sum_shopping_amt,rank
0,None,0316_OCAP他ブランド,OCAP 他ブランド利用,オンライン,3395,31707724,1
1,ＡＭＡＺＯＮ．ＣＯ．ＪＰ,0316_OCAP他ブランド,電子商取引（SSL）,オンライン,283,1378420,2
2,セブンーイレブン・ジ,0312_スーパー・コンビニ・小売り,ショッピングセンター・コンビニほか,オンライン,200,227567,3
3,None,0315_海外利用,海外,オンライン,147,1527437,4
4,イオン,0312_スーパー・コンビニ・小売り,スーパー,オンライン,119,352047,5
...,...,...,...,...,...,...,...
3929,ｈａｌｕｔａ,0316_OCAP他ブランド,電子商取引（SSL）,郵送,1,13662,1889
3930,ｉ ＬＵＭＩＮＥ,0316_OCAP他ブランド,電子商取引（SSL）,郵送,1,9000,1890
3931,ｋａｏｙａ ｐｌｕｓ,0316_OCAP他ブランド,継続的役務,郵送,1,8800,1891
3932,ｍｉｎａ ｐｅｒｈｏｎｅｎ,0312_スーパー・コンビニ・小売り,洋呉服・服地小売,郵送,1,9185,1892


In [72]:
q = """
        SELECT*
        from 
            df_shop2
        where 
            rank<=20
        order by
         channel,rank
        """

df_shop3 = sqldf(q, locals())
df_shop3

,daihyokameiten_name,meti_code,gyoshumei,channel,no_of_trn,sum_shopping_amt,rank
0,None,0316_OCAP他ブランド,OCAP 他ブランド利用,オンライン,3395,31707724,1
1,ＡＭＡＺＯＮ．ＣＯ．ＪＰ,0316_OCAP他ブランド,電子商取引（SSL）,オンライン,283,1378420,2
2,セブンーイレブン・ジ,0312_スーパー・コンビニ・小売り,ショッピングセンター・コンビニほか,オンライン,200,227567,3
3,None,0315_海外利用,海外,オンライン,147,1527437,4
4,イオン,0312_スーパー・コンビニ・小売り,スーパー,オンライン,119,352047,5
5,ファミリーマート,0312_スーパー・コンビニ・小売り,ショッピングセンター・コンビニほか,オンライン,101,79860,6
6,ETC,0316_OCAP他ブランド,サービス業,オンライン,100,162240,7
7,None,0312_スーパー・コンビニ・小売り,スーパー,オンライン,80,186802,8
8,None,0314_飲食店,料理・飲食店,オンライン,76,545637,9
9,ドラッグスギ,0312_スーパー・コンビニ・小売り,医薬品・化粧品小売,オンライン,64,167775,10


In [73]:
#CSV出力
df_shop3.to_csv(data_dir /r"output"/'new_cust_useshop_top20.csv', header=True ,index=None, encoding='shift=jis')